In [1]:
!pip install -q -U kaggle_environments

In [2]:
import random
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate
from pprint import pprint

### Бот, всегда играющий камнем

In [7]:
%%writefile rock_agent.py

def rock_agent(observation, configuration):
    return 0

Writing rock_agent.py


In [9]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[-98.0, 98.0]]

### Бот, всегда играющий бумагой

In [11]:
%%writefile paper_agent.py

def paper_agent(observation, configuration):
    return 1

Writing paper_agent.py


In [12]:
evaluate(
    "rps", #environment to use - no need to change
    ["paper_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes 
)

[[-97.0, 97.0]]

### Бот, всегда играющий ножницами

In [14]:
%%writefile scissors_agent.py

def scissors_agent(observation, configuration):
    return 2

Writing scissors_agent.py


In [15]:
evaluate(
    "rps", #environment to use - no need to change
    ["scissors_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[-99.0, 99.0]]

### Бот, всегда играющий случайным образом

In [17]:
%%writefile random_opponent.py

import random

def random_opponent(observation, configuration):
    return random.randrange(0, configuration.signs)

Writing random_opponent.py


In [18]:
evaluate(
    "rps", #environment to use - no need to change
    ["random_opponent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[0, 0]]

### Бот, никогда не играющий камнем

In [22]:
%%writefile not_rock.py

import random
def not_rock(observation, configuration):
    return random.randrange(1, configuration.signs)

Writing not_rock.py


### Бот, никогда не играющий бумагой

In [23]:
%%writefile not_paper.py

import random

def not_paper(observation, configuration):
    return random.randrange(0, configuration.signs, 2)

Writing not_paper.py


### Бот, никогда не играющий ножницами

In [25]:
%%writefile not_scissors.py

import random

def not_scissors(observation, configuration):
    return random.randrange(0, configuration.signs - 1)

Writing not_scissors.py


### Бот, повторяющий ход оппонента на прошлом ходу

In [28]:
%%writefile copy_opponent.py

import random

def copy_opponent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return random.randrange(0, configuration.signs)

Writing copy_opponent.py


### Бот, выкидывающий по очереди камень, бумагу и ножницы

In [30]:
%%writefile sequence_agent.py

counter = -1

def sequence(observation, configuration):
    global counter
    counter += 1
    if counter % 3 == 0:
        return 0
    elif counter % 3 == 1:
        return 1
    else:
        return 2

Writing sequence_agent.py


In [31]:
evaluate(
    "rps", #environment to use - no need to change
    ["sequence_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[0, 0]]

### Бот, выкидывающий равновероятно то, что не выкинул оппонент на прошлом ходу

In [33]:
%%writefile opposit_agent.py

import random

def opposit_agent(observation, configuration):
    if observation.step > 0:
        if observation.lastOpponentAction == 0:
            return random.choice([1,2])
        elif observation.lastOpponentAction == 1:
            return random.choice([0,2])
        else:
            return random.choice([1,0])
    else:
        return random.randrange(0, configuration.signs)

Writing opposit_agent.py


In [35]:
evaluate(
    "rps", #environment to use - no need to change
    ["opposit_agent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)



[[0, 0]]

### Бот, выкидывающий +1 к тому, что выкинул оппонент на прошлом ходу

In [37]:
%%writefile copynext_opponent.py

import random

def copynext_opponent(observation, configuration):
    if observation.step > 0:
        return (observation.lastOpponentAction + 1) % configuration.signs
    else:
        return random.randrange(0, configuration.signs)

Writing copynext_opponent.py


In [39]:
evaluate(
    "rps", #environment to use - no need to change
    ["copynext_opponent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)



[[59.0, -59.0]]

### Бот, выкидывающий -1 к тому, что выкинул оппонент на прошлом ходу

In [41]:
%%writefile copyprevious_opponent.py

import random

def copyprevious_opponent(observation, configuration):
    if observation.step > 0:
        return (observation.lastOpponentAction + 2) % configuration.signs
    else:
        return random.randrange(0, configuration.signs)

Writing copyprevious_opponent.py


In [42]:
evaluate(
    "rps", #environment to use - no need to change
    ["copyprevious_opponent.py", "statistical"], #agents to evaluate
    configuration={"episodeSteps": 100}, #number of episodes
)

[[-96.0, 96.0]]

# Турнир

In [44]:
agents = ['rock_agent.py',
          'paper_agent.py',
          'scissors_agent.py',
          'random_opponent.py',
          'not_rock.py',
          'not_paper.py',
          'not_scissors.py', 
          'opposit_agent.py',
          'copy_opponent.py',
          'copynext_opponent.py',
          'copyprevious_opponent.py', 
          'sequence_agent.py']

In [49]:
rank = {agent: 0 for agent in agents}

for times in range(3):
# сколько раундов сыграет каждая пара
          
    for i in range(len(agents)):
        for j in range(i+1, len(agents)):
            e = evaluate(
            "rps", #environment to use - no need to change
            [agents[i], agents[j]], #agents to evaluate
            configuration={"episodeSteps": 100, 'tieRewardThreshold': 1} #number of episodes 
            )
            if e[0][0] > e[0][1]:
                rank[agents[i]] += 1
            elif e[0][0] < e[0][1]:
                rank[agents[j]] += 1

# Итог

In [51]:
rank_se = pd.Series(rank, name="Выигранные партии каждым ботом - итоговый рейтинг").sort_values(ascending=False).to_frame()
rank_se

,Выигранные партии каждым ботом - итоговый рейтинг
copynext_opponent.py,26
not_paper.py,19
opposit_agent.py,19
scissors_agent.py,17
not_rock.py,17
random_opponent.py,16
rock_agent.py,14
not_scissors.py,13
paper_agent.py,12
copy_opponent.py,11
